In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

In [2]:
import sys, os, pathlib
import numpy as np
import xarray as xr
import torch
import matplotlib.pyplot as plt
import seaborn as sns

os.environ['DDEBACKEND'] = 'pytorch'
import deepxde

sys.path.append('..')
%aimport mre_pinn

torch.cuda.is_available()

Using backend: pytorch



True

In [ ]:
%autoreload
cohort = mre_pinn.data.ImagingCohort(['0006'])
cohort.load_images()
cohort.preprocess()
dataset = cohort.to_dataset()
dataset.save_xarrays('../data/NAFLD3')

In [3]:
example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/v3', '0006', anat=True)
example.metadata

Loading ../data/NAFLD/v3/0006/wave.nc
Loading ../data/NAFLD/v3/0006/mre.nc
Loading ../data/NAFLD/v3/0006/mre_mask.nc
Loading ../data/NAFLD/v3/0006/anat.nc
Loading ../data/NAFLD/v3/0006/anat_mask.nc


  return key in self.data



size   spacing    origin     limit    center    extent
variable dimension                                                        
wave     x           256  0.002813 -0.413808  0.303405 -0.055202  0.720026
         y           256  0.002813 -0.391718  0.325495 -0.033112  0.720026
         z             4  0.022000 -0.002113  0.063887  0.030887  0.088000
mre      x           256  0.002813 -0.413808  0.303405 -0.055202  0.720026
         y           256  0.002813 -0.391718  0.325495 -0.033112  0.720026
         z             4  0.022000 -0.002113  0.063887  0.030887  0.088000
mre_mask x           256  0.002813 -0.413808  0.303405 -0.055202  0.720026
         y           256  0.002813 -0.391718  0.325495 -0.033112  0.720026
         z             4  0.022000 -0.002113  0.063887  0.030887  0.088000
anat     x           256  0.002812 -0.413783  0.303379 -0.055202  0.719974
         y           256  0.002812 -0.391693  0.325470 -0.033112  0.719974
         z            16  0.025300 -0.158863  0.220637  0.030887  0.404801

In [4]:
example.describe()

,,dtype,count,mean,std,min,25%,50%,75%,max
variable,component,,,,,,,,,
wave,scalar,float64,262144.0,0.000139,0.023588,-0.099609,-0.004492,0.00000,0.004736,0.099609
mre,scalar,int16,262144.0,1505.871937,1335.735833,0.000000,0.000000,1340.00000,2212.000000,14078.000000
mre_mask,scalar,float32,262144.0,0.171467,0.376916,0.000000,0.000000,0.00000,0.000000,1.000000
anat,scalar,float32,5242880.0,153.163483,273.202576,-30.489124,0.015030,31.54632,206.595573,4559.377930


In [5]:
mre_pinn.baseline.eval_direct_baseline(example, frequency=40)
mre_pinn.baseline.eval_fem_baseline(example, frequency=40, hetero=True)

Evaluating direct baseline
Evaluating FEM baseline
100%|██████████| 4/4 [26:27<00:00, 396.92s/it]


In [ ]:
%autoreload
pde = mre_pinn.pde.WaveEquation.from_name('hetero', omega=40, detach=True)

In [ ]:
%autoreload
pinn = mre_pinn.model.MREPINN(
    example,
    omega=60,
    n_layers=5,
    n_hidden=128,
    polar_input=True
)
pinn

In [ ]:
%autoreload
model = mre_pinn.training.MREPINNModel(
    example, pinn, pde,
    loss_weights=[1, 0, 1, 1e-16],
    pde_warmup_iters=5000,
    pde_step_iters=5000,
    pde_init_weight=1e-18,
    n_points=1024
)
model.compile(optimizer='adam', lr=1e-4, loss=mre_pinn.training.losses.msae_loss)

In [ ]:
for t in model.data.get_raw_tensors(device='cpu', patch_size=1):
    print(t.dtype, t.shape)

In [ ]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
model.benchmark(100)

In [ ]:
test_eval = mre_pinn.testing.TestEvaluator(test_every=100, interact=True)
test_eval.model = model
test_eval.test()

In [ ]:
model.train(100000, callbacks=[test_eval])